In [4]:
from PIL import Image
import os
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator

In [5]:
# set results_folder to the folder in the same directory with the name "results"
results_folder = '../results/'
# Loop through every folder in results_folder. For each image inside the folder, add a label to the list of labels.
labels = []
for folder in os.listdir(results_folder):
    for image in os.listdir(results_folder + folder):
        labels.append(folder)
# Create a list of all the images in the results_folder
images = [os.path.join(results_folder, folder, image) for folder in os.listdir(results_folder) for image in os.listdir(results_folder + folder)]

In [6]:
# Use one hot encoding
matrix = {}
length = len(labels)
for i in range(length):
    matrix[images[i]] = to_categorical(labels.index(labels[i]), num_classes=length

In [ ]:
# Create the model
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(32, 32, 3), activation='relu'))
# Flatten
model.add(Flatten())
# Dense
model.add(Dense(128, activation='relu'))
# Output layer
model.add(Dense(10, activation='softmax'))

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer=Adam(), metrics=['accuracy'])
# Create a generator that will take the data and create batches of size 32
train_datagen = ImageDataGenerator(shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
test_datagen = ImageDataGenerator()
train_generator = train_datagen.flow_from_directory(results_folder, target_size=(200, 200), batch_size=42, class_mode='categorical')
test_generator = test_datagen.flow_from_directory(results_folder, target_size=(200, 200), batch_size=42, class_mode='categorical')

Found 273728 images belonging to 64 classes.
Found 273728 images belonging to 64 classes.


In [ ]:
# Train the model
model.fit_generator(train_generator, steps_per_epoch=42, epochs=10, validation_data=test_generator, validation_steps=42)

C:\Users\gurba\AppData\Local\Temp\ipykernel_12728\2186535113.py:2: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(train_generator, steps_per_epoch=42, epochs=10, validation_data=test_generator, validation_steps=42)


Epoch 1/10


InvalidArgumentError: Graph execution error:

Detected at node 'sequential_2/flatten_2/Reshape' defined at (most recent call last):
    File "c:\Users\gurba\AppData\Local\Programs\Python\Python38\lib\runpy.py", line 194, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "c:\Users\gurba\AppData\Local\Programs\Python\Python38\lib\runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "c:\Users\gurba\AppData\Local\Programs\Python\Python38\lib\site-packages\ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "c:\Users\gurba\AppData\Local\Programs\Python\Python38\lib\site-packages\traitlets\config\application.py", line 976, in launch_instance
      app.start()
    File "c:\Users\gurba\AppData\Local\Programs\Python\Python38\lib\site-packages\ipykernel\kernelapp.py", line 712, in start
      self.io_loop.start()
    File "c:\Users\gurba\AppData\Local\Programs\Python\Python38\lib\site-packages\tornado\platform\as